In [1]:
import pandas as pd

from sklearn.preprocessing import LabelBinarizer

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

%matplotlib inline

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('/storage/pdb/pdb_data_no_dups.csv')
df2 = pd.read_csv('/storage/pdb/pdb_data_seq.csv')
df = pd.merge(df1[['structureId','classification','macromoleculeType']], df2[['sequence','structureId']], on='structureId', how='inner')
df.head()

,structureId,classification,macromoleculeType,sequence
0,100D,DNA-RNA HYBRID,DNA/RNA Hybrid,CCGGCGCCGG
1,100D,DNA-RNA HYBRID,DNA/RNA Hybrid,CCGGCGCCGG
2,101D,DNA,DNA,CGCGAATTCGCG
3,101D,DNA,DNA,CGCGAATTCGCG
4,101M,OXYGEN TRANSPORT,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


In [3]:
df = df[df['macromoleculeType'] == 'Protein']
df.reset_index(inplace=True, drop=True)
df.head()

,structureId,classification,macromoleculeType,sequence
0,101M,OXYGEN TRANSPORT,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,102L,HYDROLASE(O-GLYCOSYL),Protein,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
2,102M,OXYGEN TRANSPORT,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
3,103L,HYDROLASE(O-GLYCOSYL),Protein,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
4,103M,OXYGEN TRANSPORT,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


In [4]:
df.shape

(110228, 4)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110228 entries, 0 to 110227
Data columns (total 4 columns):
structureId          110228 non-null object
classification       110228 non-null object
macromoleculeType    110228 non-null object
sequence             110227 non-null object
dtypes: object(4)
memory usage: 3.4+ MB


In [6]:
df.dropna(axis=0, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 110227 entries, 0 to 110227
Data columns (total 4 columns):
structureId          110227 non-null object
classification       110227 non-null object
macromoleculeType    110227 non-null object
sequence             110227 non-null object
dtypes: object(4)
memory usage: 4.2+ MB


In [7]:
df['classification'] = df['classification'].astype('str')
df = df[df['macromoleculeType'] == "Protein"]
df.reset_index(inplace=True, drop=True)

df['classification'] = df['classification'].str.lower()
df['classification'] = df['classification'].str.replace('(', '/')
df['classification'] = df['classification'].str.replace(',', '/')
df['classification'] = df['classification'].str.replace(', ', '/')
df['classification'] = df['classification'].str.replace('/ ', '/')
df['classification'] = df['classification'].str.replace(')', '')
# pattern = '|'.join([', ', ',', '('])
# df['classification'] = df['classification'].str.replace(pattern, '/')

# Maintaing class ordering. There are the same classes with different order 
#ex. viral protein/immune system and immune system/viral protein
df['classification'] = df['classification'].apply(lambda x:"/".join(sorted(x.split('/'))))

df.head(3)

,structureId,classification,macromoleculeType,sequence
0,101M,oxygen transport,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,102L,hydrolase/o-glycosyl,Protein,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
2,102M,oxygen transport,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110227 entries, 0 to 110226
Data columns (total 4 columns):
structureId          110227 non-null object
classification       110227 non-null object
macromoleculeType    110227 non-null object
sequence             110227 non-null object
dtypes: object(4)
memory usage: 3.4+ MB


In [9]:
class_dict = dict()
count = 1

classes = df['classification'].value_counts().items()

for cat, num in classes:
    
    # Remove all other classes that have number of values are less than 100
    if num < 100:
        temp = df['classification'] == cat
        df = df[~temp].copy()
        continue
        
        
        
    if num >= 100:
        class_dict[cat] = count
        count += 1

class_dict

{'hydrolase': 1,
 'transferase': 2,
 'oxidoreductase': 3,
 'lyase': 4,
 'immune system': 5,
 'structural genomics/unknown function': 6,
 'transcription': 7,
 'hydrolase/hydrolase inhibitor': 8,
 'isomerase': 9,
 'signaling protein': 10,
 'viral protein': 11,
 'transport protein': 12,
 'ligase': 13,
 'electron transport': 14,
 'membrane protein': 15,
 'toxin': 16,
 'chaperone': 17,
 'dna binding protein': 18,
 'structural protein': 19,
 'sugar binding protein': 20,
 'metal binding protein': 21,
 'protein binding': 22,
 'virus': 23,
 'unknown function': 24,
 'photosynthesis': 25,
 'contractile protein': 26,
 'cell adhesion': 27,
 'oxygen storage/transport': 28,
 'protein transport': 29,
 'rna binding protein': 30,
 'growth factor/hormone': 31,
 'hormone': 32,
 'cell cycle': 33,
 'metal transport': 34,
 'oxygen transport': 35,
 'apoptosis': 36,
 'blood clotting': 37,
 'gene regulation': 38,
 'biosynthetic protein': 39,
 'lectin': 40,
 'de novo protein': 41,
 'immunoglobulin': 42,
 'transl

In [10]:

df.head()

,structureId,classification,macromoleculeType,sequence
0,101M,oxygen transport,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,102L,hydrolase/o-glycosyl,Protein,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
2,102M,oxygen transport,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
3,103L,hydrolase/o-glycosyl,Protein,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
4,103M,oxygen transport,Protein,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93876 entries, 0 to 110226
Data columns (total 4 columns):
structureId          93876 non-null object
classification       93876 non-null object
macromoleculeType    93876 non-null object
sequence             93876 non-null object
dtypes: object(4)
memory usage: 3.6+ MB


In [12]:
df.describe()

,structureId,classification,macromoleculeType,sequence
count,93876,93876,93876,93876
unique,41940,85,1,31070
top,1GAV,hydrolase,Protein,GIVEQCCTSICSLYQLENYCN
freq,45,14858,93876,277


In [13]:
# Transform labels to one-hot
lb = LabelBinarizer()
Y = lb.fit_transform(df.classification)

In [14]:
max_length = 512 #256 # discared after 512 length
seqs = df.sequence.values
#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(seqs)
#represent input data as word rank number sequences
X = tokenizer.texts_to_sequences(seqs)
X = sequence.pad_sequences(X, maxlen=max_length)


In [15]:
lengths = [len(s) for s in seqs]
max(lengths)

2512

### 1D CNN

In [16]:
embedding_dim = 8
top_classes = 85

model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=6, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(top_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 8)            208       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 64)           3136      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 256, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 32)           6176      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 128, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128)


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 75100 samples, validate on 18776 samples
Epoch 1/20
75100/75100 [==============================] - 27s 363us/step - loss: 2.8689 - accuracy: 0.2824 - val_loss: 2.2931 - val_accuracy: 0.4254
Epoch 2/20
75100/75100 [==============================] - 27s 356us/step - loss: 1.9257 - accuracy: 0.5205 - val_loss: 1.7131 - val_accuracy: 0.5769
Epoch 3/20
75100/75100 [==============================] - 27s 360us/step - loss: 1.4240 - accuracy: 0.6463 - val_loss: 1.4208 - val_accuracy: 0.6552
Epoch 4/20
75100/75100 [==============================] - 27s 360us/step - loss: 1.1472 - accuracy: 0.7093 - val_loss: 1.2729 - val_accuracy: 0.6956
Epoch 5/20
75100/75100 [==============================] - 26s 352us/step - loss: 0.9713 - accuracy: 0.7516 - val_loss: 1.1856 - val_accuracy: 0.7211
Epoch 6/20
75100/75100 [==============================] - 27s 355us/step - loss: 0.8520 - accuracy: 0.7780 - val_loss: 1.1425 - val_accuracy: 0.7379
Epoch 7/20
75100/75100 [==============================] 

In [18]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))

train-acc = 0.9173501997336884
test-acc = 0.8066680869194717


In [110]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1), target_names=lb.classes_))

                                      precision    recall  f1-score   support

                            allergen       0.86      0.75      0.80        24
                          antibiotic       0.83      0.67      0.74        60
               antimicrobial protein       0.70      0.69      0.69        77
                           apoptosis       0.69      0.68      0.68        80
                biosynthetic protein       0.79      0.78      0.79        74
              biotin-binding protein       0.90      1.00      0.95        26
                      blood clotting       0.66      0.72      0.69        68
             calcium-binding protein       0.36      0.45      0.40        20
                       cell adhesion       0.84      0.42      0.56       144
                          cell cycle       0.76      0.49      0.60        97
                           chaperone       0.82      0.86      0.84       245
                 contractile protein       0.95      0.81      

### LSTM

In [16]:
embedding_dim = 8
lstm_out = 128
batch_size = 128
top_classes = 85

model1 = Sequential()
model1.add(Embedding(len(tokenizer.word_index)+1, 8))
model1.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(top_classes,activation='softmax'))
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 8)           208       
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               70144     
_________________________________________________________________
dense_1 (Dense)              (None, 85)                10965     
Total params: 81,317
Trainable params: 81,317
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)
model1.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_test, y_test))


Train on 75100 samples, validate on 18776 samples
Epoch 1/15
75100/75100 [==============================] - 1220s 16ms/step - loss: 3.3457 - accuracy: 0.1641 - val_loss: 3.1177 - val_accuracy: 0.2071
Epoch 2/15
75100/75100 [==============================] - 1220s 16ms/step - loss: 3.0271 - accuracy: 0.2150 - val_loss: 2.8611 - val_accuracy: 0.2544
Epoch 3/15
75100/75100 [==============================] - 1220s 16ms/step - loss: 2.8649 - accuracy: 0.2508 - val_loss: 2.7182 - val_accuracy: 0.2871
Epoch 4/15
75100/75100 [==============================] - 1219s 16ms/step - loss: 2.7318 - accuracy: 0.2788 - val_loss: 2.5822 - val_accuracy: 0.3187
Epoch 5/15
75100/75100 [==============================] - 1219s 16ms/step - loss: 2.6254 - accuracy: 0.3058 - val_loss: 2.4708 - val_accuracy: 0.3447
Epoch 6/15
75100/75100 [==============================] - 1218s 16ms/step - loss: 2.5188 - accuracy: 0.3330 - val_loss: 2.4158 - val_accuracy: 0.3675
Epoch 7/15
75100/75100 [==========================

In [21]:
train_pred = model1.predict(X_train)
test_pred = model1.predict(X_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))

train-acc = 0.5498135818908122
test-acc = 0.5304111631870473
